In [1]:
import pandas as pd

In [2]:
pincode_lookup = pd.read_csv('Pincode_lookup.csv')
pincode_lookup_or = pd.read_csv('State OR lookup.csv')

In [3]:
sales = pd.read_csv('merged_dataset_sales.csv')
orders = pd.read_csv('merged_dataset_orders.csv')

In [4]:
orders['order_id'] = orders['order_id'].str.replace('OD','',regex = False)
sales['Order Item ID'] = sales['Order Item ID'].astype(str)

In [5]:
sales = sales.rename(columns={'Order Item ID':'order_id'})

In [6]:
result = pd.merge(sales, orders, on='order_id', how='left')

In [7]:
result['order_date'].isnull().sum()

389

In [8]:
columns_remoce_double_quotes = ['Product Title/Description', 'FSN','SKU']

for column in columns_remoce_double_quotes:
    result[column] = result[column].str.replace('"""','',regex = True)

In [9]:
result['SKU'] = result['SKU'].str.replace('SKU:','',regex = True)

In [10]:
result = result.drop(['Order ID','FSN','HSN Code'], axis =1)

In [11]:
result.to_csv('merged_and_date.csv', index = False)

In [12]:
merged_dataset = result

In [13]:
merged_dataset['order_date'] = pd.to_datetime(merged_dataset['order_date'], errors = 'coerce')
merged_dataset.drop_duplicates(subset=['order_id', 'Event Type', 'Event Sub Type', ], inplace = True)

pincode_lookup_dict = pincode_lookup.set_index('Pincode')['StateName'].to_dict()
merged_dataset['Shipping State'] = merged_dataset["Customer's Delivery Pincode"].map(pincode_lookup_dict)

pincode_lookup_or['First_Numbers'] = pincode_lookup_or['First_Numbers'].astype(str).replace('\.0$','',regex = True) 
state_code_or_dict = pincode_lookup_or.set_index('First_Numbers')['State_Name'].to_dict()
merged_dataset['state'] = merged_dataset["Customer's Delivery Pincode"].astype(str).str[:2].map(state_code_or_dict)
merged_dataset['Shipping State'] = merged_dataset["Shipping State"].fillna(merged_dataset['state'])



In [14]:
null_state = merged_dataset["Customer's Billing Pincode"].where(merged_dataset['Shipping State'].isnull()).dropna(how='all')

In [15]:
merged_dataset['Gross Sale']= merged_dataset['Buyer Invoice Amount '].where((merged_dataset['Event Type']== 'Sale') & (merged_dataset['Event Sub Type'] == 'Sale'))

In [16]:
merged_dataset['Gross Sale'].fillna(0, inplace =True)

In [17]:
merged_dataset['Net Sales'] = merged_dataset['Buyer Invoice Amount '].where((merged_dataset['Event Sub Type'] != 'Return Cancellation') & (merged_dataset['Event Sub Type'] != 'Cancellation'))

In [18]:
merged_dataset['Net Sales'].fillna(0, inplace = True)

In [19]:
merged_dataset['SKU'] = merged_dataset['SKU'].replace(['"','SKU:'],'',regex = True) 
merged_dataset['Product Title/Description'] = merged_dataset['Product Title/Description'].replace('"', '', regex= True) 

In [20]:
merged_dataset['Return'] = merged_dataset['Buyer Invoice Amount '].where((merged_dataset['Event Type'] == 'Return') & (merged_dataset['Event Sub Type'] == 'Return'))
merged_dataset['Promo Sales'] = merged_dataset['Buyer Invoice Amount '].where((merged_dataset['Event Sub Type'] != 'Return Cancellation') & 
                                                                              (merged_dataset['Event Sub Type'] != 'Cancellation') & 
                                                                              (merged_dataset['Total Discount'] < 0) & (merged_dataset['Event Sub Type'] != 'Return'))

In [21]:
drop_columns = ['Seller GSTIN', 'Event Type', 'Order Type', 'Fulfilment Type','Order Approval Date ', 'Item Quantity', 'Order Shipped From (State)',
       'Price before discount', 'Total Discount', 'Seller Share ',
       'Bank Offer Share',
       'Price after discount (Price before discount-Total discount)',
       'Shipping Charges',
       'Final Invoice Amount (Price after discount+Shipping Charges)',
       'Type of tax', 'Taxable Value (Final Invoice Amount -Taxes)',
       'CST Rate', 'CST Amount', 'VAT Rate', 'VAT Amount', 'Luxury Cess Rate',
       'Luxury Cess Amount', 'IGST Rate', 'IGST Amount', 'CGST Rate',
       'CGST Amount', 'SGST Rate (or UTGST as applicable)',
       'SGST Amount (Or UTGST as applicable)', 'TCS IGST Rate',
       'TCS IGST Amount', 'TCS CGST Rate', 'TCS CGST Amount', 'TCS SGST Rate',
       'TCS SGST Amount', 'Total TCS Deducted', 'Buyer Invoice ID',
       'Buyer Invoice Date', 'Buyer Invoice Amount ', "Customer's Billing State",
       "Customer's Billing Pincode", "Customer's Delivery State",
       'Usual Price', 'Is Shopsy Order?', 'TDS Rate', 'TDS Amount']

merged_dataset.drop(drop_columns,axis = 1 ,inplace = True)

In [22]:
merged_dataset['hour'] = merged_dataset['order_date'].dt.hour

In [23]:
product_names = merged_dataset['Product Title/Description'].unique()

In [24]:
product_name_mapping  = {Product_name:f"Product {index +1}" for index, Product_name in enumerate (product_names)}

In [25]:
merged_dataset['Product Title/Description'] = merged_dataset['Product Title/Description'].replace(product_name_mapping)

In [26]:
merged_dataset.drop(['order_id','SKU','Warehouse ID','state'],axis = 1, inplace = True)

In [27]:
fill_na_columns = ['Net Sales', 'Return','Promo Sales']

for column in fill_na_columns:
    merged_dataset[column] = merged_dataset[column].fillna(0)

In [28]:
merged_dataset.to_csv('final_dataset.csv',index = False)

In [29]:
merged_dataset

,Product Title/Description,Event Sub Type,Order Date,Customer's Delivery Pincode,order_date,Shipping State,Gross Sale,Net Sales,Return,Promo Sales,hour
0,Product 1,Sale,2023-08-29 00:00:00.0,226002,2023-08-29 13:44:01,Uttar Pradesh,904.0,904.0,0.0,0.0,13.0
1,Product 2,Sale,2023-09-01 00:00:00.0,534462,2023-09-01 18:50:10,Andhra Pradesh,234.0,234.0,0.0,0.0,18.0
2,Product 2,Return,2023-08-26 00:00:00.0,600045,2023-08-26 10:58:44,Tamil Nadu,0.0,-222.0,-222.0,0.0,10.0
3,Product 3,Return,2023-08-23 00:00:00.0,335524,2023-08-23 12:36:51,Rajasthan,0.0,-234.0,-234.0,0.0,12.0
4,Product 3,Sale,2023-09-01 00:00:00.0,500035,2023-09-01 16:13:03,Telangana,222.0,222.0,0.0,0.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
4305,Product 2,Sale,2023-07-30 00:00:00.0,533274,2023-07-30 10:34:26,Andhra Pradesh,229.0,229.0,0.0,0.0,10.0
4306,Product 2,Sale,2023-07-29 00:00:00.0,533001,2023-07-29 20:01:39,Andhra Pradesh,234.0,234.0,0.0,0.0,20.0
4307,Product 25,Return,2023-07-23 00:00:00.0,533254,2023-07-23 18:19:32,Andhra Pradesh,0.0,-1509.0,-1509.0,0.0,18.0
4308,Product 25,Sale,2023-07-29 00:00:00.0,141119,2023-07-29 10:40:51,Punjab,1409.0,1409.0,0.0,1409.0,10.0
